# Table Creation for Crypto Analytics Platform

This notebook creates all tables in the Neon Postgres database for the crypto market intelligence platform.

In [ ]:
# Setup and Imports
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine, text

load_dotenv()

DATABASE_URL = os.getenv("DATABASE_URL")
print("Database URL loaded:", DATABASE_URL is not None)

engine = create_engine(DATABASE_URL)

In [ ]:
# Enable UUID Extension
with engine.begin() as conn:
    conn.execute(text('CREATE EXTENSION IF NOT EXISTS "uuid-ossp";'))
    print("UUID extension enabled")

In [ ]:
# Create crypto_group table
sql = text("""
CREATE TABLE IF NOT EXISTS crypto_group (
    id          UUID PRIMARY KEY DEFAULT uuid_generate_v4(),
    tag         TEXT NOT NULL UNIQUE,
    type        TEXT NOT NULL,
    description TEXT
);
""")

with engine.begin() as conn:
    conn.execute(sql)
    print("✓ crypto_group table created")

In [ ]:
# Create crypto_asset table
sql = text("""
CREATE TABLE IF NOT EXISTS crypto_asset (
    id                      UUID PRIMARY KEY DEFAULT uuid_generate_v4(),
    coingecko_id            TEXT NOT NULL UNIQUE,
    symbol                  VARCHAR(20) NOT NULL,
    name                    VARCHAR(100) NOT NULL,
    is_active               BOOLEAN NOT NULL DEFAULT TRUE,
    time_added              TIMESTAMPTZ NOT NULL DEFAULT NOW(),
    last_daily_observed_at  TIMESTAMPTZ,
    last_hourly_observed_at TIMESTAMPTZ
);

CREATE INDEX IF NOT EXISTS idx_crypto_asset_symbol ON crypto_asset(symbol);
""")

with engine.begin() as conn:
    conn.execute(sql)
    print("✓ crypto_asset table created")

In [ ]:
# Create crypto_asset_group bridge table
sql = text("""
CREATE TABLE IF NOT EXISTS crypto_asset_group (
    asset_id UUID NOT NULL REFERENCES crypto_asset(id) ON DELETE CASCADE,
    group_id UUID NOT NULL REFERENCES crypto_group(id) ON DELETE CASCADE,
    PRIMARY KEY (asset_id, group_id)
);

CREATE INDEX IF NOT EXISTS idx_crypto_asset_group_group_id ON crypto_asset_group(group_id);
""")

with engine.begin() as conn:
    conn.execute(sql)
    print("✓ crypto_asset_group bridge table created")

In [ ]:
# Create crypto_asset_price_daily fact table
sql = text("""
CREATE TABLE IF NOT EXISTS crypto_asset_price_daily (
    id             BIGSERIAL PRIMARY KEY,
    asset_id       UUID NOT NULL REFERENCES crypto_asset(id) ON DELETE CASCADE,
    observed_at    TIMESTAMPTZ NOT NULL,
    currency_code  CHAR(3) NOT NULL DEFAULT 'USD',
    price          NUMERIC(18,8) NOT NULL,
    market_cap_usd NUMERIC(20,4),
    volume_24h_usd NUMERIC(20,4),
    UNIQUE (asset_id, observed_at, currency_code)
);

CREATE INDEX IF NOT EXISTS idx_daily_asset_id ON crypto_asset_price_daily(asset_id);
CREATE INDEX IF NOT EXISTS idx_daily_observed_at ON crypto_asset_price_daily(observed_at);
""")

with engine.begin() as conn:
    conn.execute(sql)
    print("✓ crypto_asset_price_daily fact table created")

In [ ]:
# Create crypto_asset_price_hourly fact table
sql = text("""
CREATE TABLE IF NOT EXISTS crypto_asset_price_hourly (
    id             BIGSERIAL PRIMARY KEY,
    asset_id       UUID NOT NULL REFERENCES crypto_asset(id) ON DELETE CASCADE,
    observed_at    TIMESTAMPTZ NOT NULL,
    currency_code  CHAR(3) NOT NULL DEFAULT 'USD',
    price          NUMERIC(18,8) NOT NULL,
    market_cap_usd NUMERIC(20,4),
    volume_24h_usd NUMERIC(20,4),
    UNIQUE (asset_id, observed_at, currency_code)
);

CREATE INDEX IF NOT EXISTS idx_hourly_asset_id ON crypto_asset_price_hourly(asset_id);
CREATE INDEX IF NOT EXISTS idx_hourly_observed_at ON crypto_asset_price_hourly(observed_at);
""")

with engine.begin() as conn:
    conn.execute(sql)
    print("✓ crypto_asset_price_hourly fact table created")

In [ ]:
# Create job_run_log operational table
sql = text("""
CREATE TABLE IF NOT EXISTS job_run_log (
    job_name    TEXT PRIMARY KEY,
    last_run_at TIMESTAMPTZ,
    last_status TEXT,
    details     JSONB
);
""")

with engine.begin() as conn:
    conn.execute(sql)
    print("✓ job_run_log operational table created")

In [ ]:
# Verify all tables created
sql = text("""
SELECT table_name 
FROM information_schema.tables 
WHERE table_schema = 'public' 
ORDER BY table_name;
""")

with engine.begin() as conn:
    result = conn.execute(sql)
    tables = [row[0] for row in result.fetchall()]
    print("\nTables in database:")
    for table in tables:
        print(f"  - {table}")

In [ ]:
# New: crypto_asset_group_history
# Purpose: Track when assets enter and exit groups over time
# Populated by: GroupSelector.py when group membership changes

sql = text("""
CREATE TABLE IF NOT EXISTS crypto_asset_group_history (
    id               BIGSERIAL PRIMARY KEY,
    asset_id         UUID NOT NULL REFERENCES crypto_asset(id) ON DELETE CASCADE,
    group_id         UUID NOT NULL REFERENCES crypto_group(id) ON DELETE CASCADE,
    event_type       VARCHAR(10) NOT NULL CHECK (event_type IN ('JOINED', 'LEFT')),
    event_timestamp  TIMESTAMPTZ NOT NULL DEFAULT NOW(),
    market_cap_usd   NUMERIC(20,4),
    rank_in_group    INTEGER,
    metadata         JSONB
);

CREATE INDEX IF NOT EXISTS idx_group_history_asset_id ON crypto_asset_group_history(asset_id);
CREATE INDEX IF NOT EXISTS idx_group_history_group_id ON crypto_asset_group_history(group_id);
CREATE INDEX IF NOT EXISTS idx_group_history_event_timestamp ON crypto_asset_group_history(event_timestamp);
CREATE INDEX IF NOT EXISTS idx_group_history_asset_group ON crypto_asset_group_history(asset_id, group_id);
""")

with engine.begin() as conn:
    conn.execute(sql)
    print("✓ crypto_asset_group_history table created")
    print("\nTable details:")
    print("  - Tracks: When assets join/leave groups")
    print("  - Event types: JOINED, LEFT")
    print("  - Captures: Market cap and rank at time of event")
    print("  - Metadata: JSONB for additional context (e.g., reason for change)")